[XML이란?](http://mommoo.tistory.com/17)  
[점프 투 파이썬](https://wikidocs.net/42#_2)  
[Elements and Element Trees](http://effbot.org/zone/element.htm)  
하기 내용은 상기 링크를 정리한것.  

# XML

## XML이란?
XML(eXtensible Markup Language)는 확장될수 있는 표시 언어라고 생각하면된다.
대표적인 Markup Language가 HTML이다.  
XML과 HTML의 큰 차이점은.
* HTML
    * 이미 약속한 태그들만 사용가능.  
    ex) ```<h1></h1>```=>글자크기를 키우는 태그.  
    즉 이미 만들어서 제공되는 태그만 사용해야한다. 
  
* XML
    * 사용자 임의대로 만들어 사용가능하다.  
    즉, XML은 어떠한 데이터를 설명하기 위해 이름을 임의로 지은 태그로 데이터를 감싼다.  
    다시말해서 태그로 데이터를 설명하는 것이다.  
    이 부분에서 데이터의 표시(Markup)가 되고,  
    더 필요한 데이터가 생길시 태그추가나,  
    태그안의 내용을 추가할수 있다는 특징이 있다.

## 파이썬으로 XML 처리하기

이 절에서는 파이썬으로 XML 문서를 다루는 방법에 대해서 살펴본다.  
XML 처리를 위한 파이썬 라이브러리는 아주 많으며 아래 사이트에서 확인가능.  
[http://wiki.python.org/moin/PythonXml](http://wiki.python.org/moin/PythonXml)

### XML 문서 생성하기

```
<note date = '20120104'>
    <to>Tove</To>
    <from>Jani</from>
    <heading>Reminder</heading>
    <body>Dont't forget me this weekend!</body>
</note>
```

In [2]:
from xml.etree.ElementTree import Element, dump

note = Element("note")
to = Element("to")
to.text = 'Tove'

note.append(to)
dump(note)

<note><to>Tove</to></note>


위의 코드를 작성하면 결과물이 출력된다.  
Element를 이용하여태그를 생성하고, 만들어진 태그에 텍스트 값을 추가 할수 있다.

#### SubElement

In [3]:
from xml.etree.ElementTree import Element, SubElement, dump

# note에 tag를 추가하는 방법1
note = Element("note")
to = Element("to")
to.text = "Tove"
note.append(to)

# note에 tag를 추가하는 방법2
SubElement(note, "from").text = "Jani"

dump(note)

<note><to>Tove</to><from>Jani</from></note>


SubElement는 태그명과 태그의 텍스트 값을 한 번에 설정할 수 있다.  
또는 다음과 같이 태그 사이에 태그를 추가하거나 특정태그를 삭제할 수 도 있다.

<br>

dummy 태그 삽입

In [11]:
dummy = Element("dummy")
note.insert(1, dummy)
dump(note)

<note><to>Tove</to><dummy /><dummy /><from>Jani</from></note>


dummy 태그 삭제

In [12]:
note.remove(dummy)
dump(note)

<note><to>Tove</to><dummy /><from>Jani</from></note>


### Attribute 추가하기
이번에는 note태그에 Attribute를 추가한다.

In [13]:
from xml.etree.ElementTree import Element, SubElement, dump

note = Element("note")
to = Element("to")
to.text = "Tove"
note.append(to)

SubElement(note, "from").text = "Jani"

note.attrib['date'] = "20120104"

dump(note)

<note date="20120104"><to>Tove</to><from>Jani</from></note>


note.attrib['date']와 같은 방법으로 Attribute value를 추가 할 수 있다.  
다음과 같이 엘리먼트 생성 시 직접 애트리뷰트 값을 추가하는 방법을 사용해도 된다.  

In [14]:
note = Element("note", date = '20120104')

아래는 Attribute를 이용한 완성된 소스이다.

In [15]:
from xml.etree.ElementTree import Element, SubElement, dump

note = Element('note')
note.attrib['date'] = '20120104'

to = Element('to')
to.text = "Tove"
note.append(to)

SubElement(note, 'from').text = "Jani"
SubElement(note, 'heading').text = "Reminder"
SubElement(note, 'body').text = "Don't forget me this weekend!"

dump(note)

<note date="20120104"><to>Tove</to><from>Jani</from><heading>Reminder</heading><body>Don't forget me this weekend!</body></note>


### indent 함수

위의 결과 값은 한줄로 작성되어 보기가 쉽지 않다.  
해서 정렬된 형태의 XML값을 보려면 다음과 같이 indent 함수를 작성해본다.

In [17]:
from xml.etree.ElementTree import Element, SubElement, dump

note = Element('note')
note.attrib['date'] = '20120104'

to = Element('to')
to.text = "Tove"
note.append(to)

SubElement(note, "from").text = "Jani"
SubElement(note, "heading").text = "Reminder"
SubElement(note, "body").text = "Don't forget me this weekend!"

def indent(elem, level=0):
    i = "\n" + level*"    "
    if len(elem):
        if not elem.text or not elem.text.strip():
            elem.text = i + "    "
        if not elem.tail or not elem.tail.strip():
            elem.tail = i
        for elem in elem:
            indent(elem, level+1)
        if not elem.tail or not elem.tail.strip():
            elem.tail = i
    else:
        if level and (not elem.tail or not elem.tail.strip()):
            elem.tail = i

indent(note)
dump(note)

<note date="20120104">
    <to>Tove</to>
    <from>Jani</from>
    <heading>Reminder</heading>
    <body>Don't forget me this weekend!</body>
</note>


### 파일에 쓰기(write) 수행하기

이제 생성한 xml 문서를 가지고 다음과 같이 element의 write메서드를 이용하서 파일을 써본다.

In [20]:
from xml.etree.ElementTree import ElementTree
ElementTree(note).write("note.xml")

이제 xml이 생성된것을 볼 수 있다.

### XML문서 파싱하기

In [22]:
from xml.etree.ElementTree import parse
tree = parse("note.xml")
note = tree.getroot()

#### 애트리뷰터 값 읽기

In [23]:
print(note.get("data"))
print(note.get("foo", "default"))
print(note.keys())
print(note.items())

None
default
['date']
[('date', '20120104')]


#### XML 태그 접근하기

In [26]:
from_tag = note.find("from")
from_tags = note.findall("from")
from_text = note.findtext("from")

note.find("from")은 note 태그 하위에 from과 일치하는 첫 번째 태그를 찾아서 리턴하고, 없으면 None을 리턴한다.  
note.findall("from")은 note 태그 하위에 from과 일치하는 모든 태그를 리스트로 리턴한다.  
note.findtext("from")은 note 태그 하위에 from과 일치하는 첫번째 태그의 텍스트 값을 리턴한다.  
  
특정 태그의 모든 하위 엘리먼트를 순차적으로 처리할 때는 아래의 메서드를 사용한다.

In [27]:
childs = note.getiterator()
childs = note.getchildren()

gettiterator()함수는 첫 번째 인수로 다음과 같이 태그명을 받을 수도 있다.

In [28]:
note.getiterator("from")

위와 같은 경우 from 태그의 하위 엘리먼트들이 순차적으로 리턴되며, 보통다음과 같이 많이 사용된다.

In [29]:
for parent in terr.getiterator():
    for child in parent:
        ... work on parent/child tuple

SyntaxError: invalid syntax (<ipython-input-29-37a7b31559aa>, line 3)

위의 코드나 더 자세한 내용은 프레드릭 런드가 직접 작성한 튜토리얼에서 확인하면된다.
[http://effbot.org/zone/element.htm](http://effbot.org/zone/element.htm)